
### Packages


In [32]:
import cv2 as cv
import cv2
import matplotlib.pyplot as plt
import os
import numpy as np
from preprocess import *
import pandas as pd

In [14]:
for image in os.listdir('images'):
    if image.endswith('jpg'):
        img = cv.imread(os.path.join('images', image))
        img = crop_to_square_and_resize(img, 256)
        img = equalize(img)
        seg = extract_bv(img)
        if (sum(sum(seg/255))) < 6e3: continue
        img = color_correct(img, 30)
        extracted = cv.bitwise_and(img, img, mask = seg)
        cv.imwrite(os.path.join('images_prepro', image), extracted)


In [72]:
from packages.small_batch_images import *
df = load_annotation()
healthy_labels = pd.DataFrame(columns=['age', 'mean_rx', 'mean_ry', 'var_rx', 'var_ry',\
                                'mean_bx', 'mean_by', 'var_bx', 'var_by',\
                                'mean_gx', 'mean_gy', 'var_gx', 'var_gy',\
                                'mean_r', 'mean_g', 'mean_b', \
                                'var_r', 'var_g', 'var_b'])

unhealthy_labels = healthy_labels.copy()

def calculate_values(image):
    img = cv.imread(image)
    B, G, R = cv.split(img)
    R_coord = cv.findNonZero(R).squeeze()
    G_coord = cv.findNonZero(G).squeeze()
    B_coord = cv.findNonZero(B).squeeze()
    return {
            'mean_rx': R_coord.mean(axis=0)[0], 'mean_ry':R_coord.mean(axis=0)[1], \
            'var_rx':R_coord.var(axis=0)[0], 'var_ry':R_coord.var(axis=0)[0],\
            'mean_bx': B_coord.mean(axis=0)[0], 'mean_by':B_coord.mean(axis=0)[1], \
            'var_bx':B_coord.var(axis=0)[0], 'var_by':B_coord.var(axis=0)[0],\
            'mean_gx': G_coord.mean(axis=0)[0], 'mean_gy':G_coord.mean(axis=0)[1], \
            'var_gx':G_coord.var(axis=0)[0], 'var_gy':G_coord.var(axis=0)[0],\
            'mean_r':R.mean(), 'mean_g':G.mean(), 'mean_b':B.mean(), \
            'var_r':R.var(), 'var_g':G.var(), 'var_b':B.var()
    }

i = 0; j = 0
for ind in df.index:
    row = df.loc[ind]
    # healthy/unhealthy set construction
    if os.path.isfile('images_prepro/{}'.format(row.Left_Fundus)):
        if row.Left_Diagnosis == 'normal fundus':
            healthy_labels.loc[i] = {'age': row['Age'],\
                 **calculate_values('images_prepro/{}'.format(row.Left_Fundus))}
            i+=1
        else:
            unhealthy_labels.loc[j] = {'age': row['Age'],\
                 **calculate_values('images_prepro/{}'.format(row.Left_Fundus))}
            j+=1
    if os.path.isfile('images_prepro/{}'.format(row.Right_Fundus)):
        if row.Right_Diagnosis == 'normal fundus':
            healthy_labels.loc[i] = {'age': row['Age'], \
                 **calculate_values('images_prepro/{}'.format(row.Right_Fundus))}
            i+=1
        else:
            unhealthy_labels.loc[j] = {'age': row['Age'], \
                 **calculate_values('images_prepro/{}'.format(row.Right_Fundus))}
            j+=1

In [70]:
healthy_labels.head()

,age,mean_rx,mean_ry,var_rx,var_ry,mean_bx,mean_by,var_bx,var_by,mean_gx,mean_gy,var_gx,var_gy,mean_r,mean_g,mean_b,var_r,var_g,var_b
0,57,130.913887,129.604170,2659.823528,2659.823528,129.941790,130.627000,2679.568040,2679.568040,128.167904,129.161683,2706.470838,2706.470838,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,66,117.741195,103.150979,4025.802419,4025.802419,118.210883,103.844510,4345.980007,4345.980007,118.165266,103.991660,4185.325970,4185.325970,"[0.28125, 0.23046875, 0.31640625, 0.28515625, ...","[0.2265625, 0.23828125, 0.15234375, 0.25390625...","[0.53125, 0.55078125, 1.0703125, 1.90625, 3.37...","[0.6083984375, 0.5601654052734375, 0.638168334...","[0.38616943359375, 0.4158782958984375, 0.30882...","[1.8662109375, 1.9817962646484375, 7.799743652..."
2,59,112.714115,123.881562,3395.144986,3395.144986,114.678232,125.027465,3542.114778,3542.114778,114.817993,125.643631,3602.407791,3602.407791,"[0.37890625, 0.40234375, 10.89453125, 9.796875...","[0.46484375, 1.0703125, 14.1640625, 13.1796875...","[1.90234375, 2.296875, 14.6640625, 14.8359375,...","[1.2040863037109375, 1.7326507568359375, 1514....","[0.9518890380859375, 5.61224365234375, 2206.23...","[29.759994506835938, 35.818115234375, 1781.801..."
3,59,137.907144,128.087083,4081.215574,4081.215574,138.128934,127.000300,4080.620040,4080.620040,137.342082,128.091947,4051.115418,4051.115418,"[0.46484375, 0.8046875, 14.4765625, 16.5351562...","[0.5, 0.63671875, 13.8671875, 15.44921875, 18....","[2.5234375, 2.984375, 16.37890625, 18.85546875...","[1.2643890380859375, 4.82904052734375, 2164.64...","[3.4375, 5.0047454833984375, 1963.958923339843...","[48.53851318359375, 55.484130859375, 1933.8290..."
4,54,127.742664,125.344784,3174.634423,3174.634423,132.227784,124.561987,3343.838706,3343.838706,128.578066,126.240630,3177.792432,3177.792432,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.031...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.031...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.031...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.061...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.061...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.061..."


In [81]:
# Use stratified method to select samples, leave 20% for testing, together with unhealthy ones
train = healthy_labels.groupby('age', group_keys=False).apply(lambda x: x.sample(min(len(x), 10)))
test = healthy_labels[~healthy_labels.index.isin(train.index)]
test = pd.concat((test, unhealthy_labels.sample(200)))
(len(train), len(test))

(469, 1125)

In [82]:
from sklearn.linear_model import LinearRegression
lr_model = LinearRegression()
train = train.to_numpy()
test = test.to_numpy()
lr_model.fit(train[:, 1:], train[:, 0])
# R2 score
lr_model.score(train[:, 1:], train[:, 0])

0.12089113145071662

In [83]:
# R2 score
lr_model.score(test[:, 1:], test[:, 0])

-0.2739937006882285